# wiz.io Reporting tool

## Establishing the connection

The first step is log on to the [wiz.io portal](https://app.wiz.io/) and obtain an API token. You will need it below.

In [21]:
import pandas as pd
import requests
from datetime import date

# Colors for plotting
colors = {"info": "#0000ff",
          "low": "#00ff00",
          "medium": "#ffff00",
          "high": "#ff8000",
          "critical": "#ff0000"}

# reusable functions
def query_wiz_api(token, query, variables):  
  """Query WIZ API for the given query data schema"""

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
  }

  data = {"variables": variables, "query": query}

  try:
      result = requests.post(url="https://api.us8.app.wiz.io/graphql", json=data, headers=headers)

  except Exception as e:
      if '502: Bad Gateway' not in str(e) and '503: Service Unavailable' not in str(e):
          print(f"Wiz-API-Error: {str(e)}")
          return(e)
      else:
          print("Retry")

  return result.json()

def request_wiz_api_token(client_id, client_secret):  
  """Retrieve an OAuth access token to be used against Wiz API"""

  headers = {"Content-Type": "application/x-www-form-urlencoded"}

  auth_payload = {
    'grant_type': 'client_credentials',
    'audience': 'beyond-api',
    'client_id': client_id,
    'client_secret': client_secret
  }

  response = requests.post("https://auth.wiz.io/oauth/token", headers=headers, data=auth_payload)

  if response.status_code != requests.codes.ok:
      raise Exception(f"Error authenticating to Wiz [{response.status_code}] - {response.text}")

  try:
      response_json = response.json()
      token = response_json.get('access_token')
      if not token:
          message = f"Could not retrieve token from Wiz: {response_json.get('message')}"
          raise Exception(message)
  except ValueError as exception:
      print(exception)
      raise Exception("Could not parse API response")

  return token

#token = request_wiz_api_token(client_id, client_secret)
token = input("Enter your wiz.ui API Token")


## Issue count (Severity)

The following table displays the total vulnerability count grouped by Severity.

In [ ]:
query = ("""
  query IssueSeverity($issueFilter: IssueFilters) {
    issues(filterBy: $issueFilter) {
      info: informationalSeverityCount
      low: lowSeverityCount
      medium: mediumSeverityCount
      high: highSeverityCount
      critical: criticalSeverityCount
    }
  }
""")
variables = {
  "issueFilter": {
#      "project": "5aa1d4a6-67dc-5466-899b-8e7c001db5b7",
      "status": ["OPEN", "IN_PROGRESS"]
    }
}
result = query_wiz_api(token, query, variables)
df = pd.DataFrame([result["data"]["issues"]]).transpose().reset_index().rename(columns={"index":"Severity", 0:"Count"})
df.to_csv(f"vulnerabilities-{date.today()}.csv", encoding="utf-8", index=False, sep=";")
df.plot(kind='pie', y='Count', autopct='%1.1f%%', labels=df['Severity'], colors=colors.values())
df

## Issue break down (Cloud Account/Severity)

The following table displays the total count of vulnerabilities grouped by Cloud Account and Severity.

In [ ]:
cloudAccountsQuery = ("""
query CloudAccounts($first: Int, $cloudAccountFilter: CloudAccountFilters, $issueFilter: IssueFilters) {
  cloudAccounts(first: $first, filterBy: $cloudAccountFilter) {
    nodes {
      accountName: name
      provider: cloudProvider
      status
      lastScan: lastScannedAt
      resources: resourceCount
      issues(filterBy: $issueFilter) {
        info: informationalSeverityCount
        low: lowSeverityCount
        medium: mediumSeverityCount
        high: highSeverityCount
        critical: criticalSeverityCount
      }
    }
  }
}
""")
variables = {
  "first": 100,
  "cloudAccountFilter": {
#      "projectId": "5aa1d4a6-67dc-5466-899b-8e7c001db5b7",
      "status": ["CONNECTED", "PARTIALLY_CONNECTED"]
    },
  "issueFilter": {"status": ["OPEN", "IN_PROGRESS"]}
}
cloudAccounts = query_wiz_api(token, cloudAccountsQuery, variables)
data = [ 
  {
    "accountName": d["accountName"],
    "provider": d["provider"],
    "status": d["status"],
    "lastScan": d["lastScan"],
    "info": d["issues"]["info"],
    "low": d["issues"]["low"],
    "medium": d["issues"]["medium"],
    "high": d["issues"]["high"],
    "critical": d["issues"]["critical"],
  } for d in cloudAccounts["data"]["cloudAccounts"]["nodes"]
 ]
df = pd.DataFrame(data).sort_values(by=["critical", "high", "medium", "low", "info"], ascending=False)
df.to_csv(f"cloudaccounts-{date.today()}.csv", encoding="utf-8", index=False, sep=";")
from matplotlib import pyplot as plt
df.plot(kind='barh', x="accountName", stacked=True, color=colors.values())
plt.title("Vulnerability Break Down")
plt.xlabel("Count")
plt.ylabel("Cloud Account")
df